In [1]:
import numpy as np
import xarray as xr
from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings('ignore')

# from multiprocessing import Pool  ### module for parallel programming
import time as time_module ## module to time this cell
import gc

In [2]:
# year_list = ['2001']
year_list = np.char.zfill(np.arange(2001, 2019+1).astype(str), 4)

In [3]:
year_list

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'], dtype='<U4')

In [4]:
mon_list = ['06','07','08']
# mon_list = ['12','01','02']
# mon_list = ['03','04','05']
# mon_list = ['09','10','11']

In [5]:
RGMA_dir = '/neelin2020/RGMA_feature_mask/data_product/'
for year in year_list:
    for mon in mon_list:
        datadir = RGMA_dir+year+'/MERGED_FP/'
        filename = 'GPM_feature_merged_'+mon+'_v4.nc'
        ds_monyear = xr.open_mfdataset('%s/%s'%(datadir, filename), engine='netcdf4', combine='nested', concat_dim='time')
        if (year == year_list[0]) & (mon == mon_list[0]):
            ds_monall = ds_monyear.copy()
        else:
            ds_monall = xr.concat([ds_monall, ds_monyear], dim='time', join='exact')  
ds_monall = ds_monall.rename({'latitude':'lat', 'longitude':'lon'})
ds_monall = ds_monall.drop_sel(lon=360)         

In [6]:
ds_monall

<xarray.Dataset>
Dimensions:           (time: 6992, lon: 1440, lat: 481)
Coordinates:
  * time              (time) datetime64[ns] 2001-06-01 ... 2019-08-31T18:00:00
  * lon               (lon) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * lat               (lat) float64 -60.0 -59.75 -59.5 ... 59.5 59.75 60.0
Data variables:
    precipitationCal  (time, lat, lon) float32 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    ar_tag            (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    front_c_tag       (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    front_w_tag       (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    front_s_tag       (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    lps_tag           (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>
    mcs_tag           (time, lat, lon) int8 dask.array<chunksize=(120, 481, 1440), meta=np.ndarray>

In [7]:
time1 = ds_monall.time

In [8]:
mon_list,len(time1)

(['06', '07', '08'], 6992)

In [9]:
Precip = ds_monall['precipitationCal']


In [10]:
lat = Precip.lat
lon = Precip.lon
lat_coarse = lat.coarsen(lat=8*5, boundary="pad").mean().rename({'lat':'lat_coarse'})
lon_coarse = lon.coarsen(lon=8*5, boundary="pad").mean().rename({'lon':'lon_coarse'})

In [11]:
Precip_coarse = Precip.coarsen(lat=8*5, lon=8*5, boundary="pad").construct(lon=("lon_coarse", "lon_fine"), lat=("lat_coarse", "lat_fine")).stack(fine_gridstime=("lon_fine", "lat_fine", "time"))
# FT_mask_coarse = FT_mask.coarsen(lat=8*5, lon=8*5, boundary="pad").construct(lon=("lon_coarse", "lon_fine"), lat=("lat_coarse", "lat_fine")).stack(fine_gridstime=("lon_fine","lat_fine","time"))


In [12]:
# bins in log space to calculate the PDFs
bll_Xlog = np.logspace(np.log(0.02), np.log(150), 100, base=np.e)
binplot_Xlog = (bll_Xlog[:-1]+bll_Xlog[1:])/2

# insert a first bin to host values near zero
bll_Xlog1 = np.insert(bll_Xlog, 0, 1e-3)
binplot_Xlog1 = np.insert(binplot_Xlog, 0, 1e-3)


In [13]:
def PDFs(X, bll_X=np.linspace(0, 150, 100)):  
    '''
    Function to calculate PDFs
    Input   X: Precip
            bll_X: bins for Precip
    Output  hist_X: probability density
            histN_X: frequency distribution

    '''

    if np.any(X.mask):
        X_use = X[~X.mask]
    else:
        X_use = X.flatten()

    hist_X, bin_edges = np.histogram(X_use, bins=bll_X, density=True)
    histN_X, bin_edges = np.histogram(X_use, bins=bll_X, density=False)
    # binplot_X = (bin_edges[:-1]+bin_edges[1:])/2.
    
    del X_use # delete the working variable to same memory space
    
    return hist_X, histN_X


In [14]:
def fn(x, a, b, c):
    return a + b*x[0] + c*x[1]


In [15]:
def PDFs_Params_Log(Precip_feature, lat_coarse=lat_coarse, lon_coarse=lon_coarse, bll_Xlog=bll_Xlog1):

    '''
    Function that returns: (i) spatial fields of precipitation PDFs, (ii) fitted P_L and tau, and (iii) the fitted curves.

    '''

    histLogbin_Precip_feature = np.full((len(lat_coarse),len(lon_coarse),len(bll_Xlog)-1), np.nan)
    histNLogbin_Precip_feature = np.full((len(lat_coarse),len(lon_coarse),len(bll_Xlog)-1), np.nan)
    fit_tau_feature = np.full((len(lat_coarse),len(lon_coarse)), np.nan)
    fit_PL_feature = np.full((len(lat_coarse),len(lon_coarse)), np.nan)
    Y_fit_feature =  np.full(histLogbin_Precip_feature.shape, np.nan)
    X_fit_feature =  np.full(histLogbin_Precip_feature.shape, np.nan)

    bincenters_Xlog = (bll_Xlog[1:-1]+bll_Xlog[2:])/2      # for fitting; exclude the first bin that hosts near-zeros; take the center of the bin edges
    
    x1 = np.log(bincenters_Xlog)    # x1 = log(precip)
    x2 = bincenters_Xlog           # x2 = precip

    for ii in range(len(lat_coarse)):
        for jj in range(len(lon_coarse)):
            X = Precip_feature[ii,jj,:]
            # X = np.ma.masked_less(Precip_feature, 0.1)
            histLogbin_Precip_feature[ii,jj,:], histNLogbin_Precip_feature[ii,jj,:] = PDFs(X, bll_Xlog)
            
            hist_work = np.ma.masked_invalid(histLogbin_Precip_feature[ii,jj,1:])  # exclude the 1st bin that host counts for non-raining conditions
            # histN_work = np.ma.masked_invalid(histNLogbin_Precip_feature[ii,jj,1:])  # counts, to mask bins with too small sample size
            # hist_work = np.ma.masked_where(abs(hist_work)<1e-5, hist_work)    # do not include bins with a density value lower than xxx
            # hist_work = np.ma.masked_where(histN_work<10, hist_work)    # do not include bins with count number lower than xxx
            
            
            if (~np.all(hist_work.mask)):   # only operate on grid cells with value
                y = np.log(hist_work) # y = log(pdf)
                y_ma = np.ma.masked_invalid(y)  # filter out bins with no counts
                yy = y_ma[~y_ma.mask]
                xx1= x1[~y_ma.mask]
                xx2 = x2[~y_ma.mask]
                xx = np.stack((xx1, xx2))

                if len(yy) > 10:
                    popt, pcov = curve_fit(fn, xx, yy)   
                else:
                    popt = np.full(3, np.nan)

                fit_tau_feature[ii,jj] = -1*popt[1]
                fit_PL_feature[ii,jj] = -1/popt[2]

                ff1 = fn(xx, popt[0], popt[1], popt[2])   # this is log(pdf), exp(ff1) is the pdf values
                Y_fit_feature[ii,jj,1:len(xx[1])+1] = ff1
                X_fit_feature[ii,jj,1:len(xx[1])+1] = xx[1]        # x2 = precip

    return  histLogbin_Precip_feature, histNLogbin_Precip_feature, fit_tau_feature, fit_PL_feature, X_fit_feature, Y_fit_feature




In [16]:
Precip_total = np.ma.masked_invalid(Precip_coarse)    


In [17]:
start_time = time_module.time()
histLogbin_Precip_total, histNLogbin_Precip_total, fit_tau_total, fit_PL_total, X_fit_total, Y_fit_total = PDFs_Params_Log(Precip_total)
print('Took %.2f minutes'%((time_module.time()-start_time)/60))        

da_histLogbin_Precip_total = xr.DataArray(histLogbin_Precip_total, dims=('lat_coarse','lon_coarse','Xbin'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse,'Xbin':binplot_Xlog1}, name='histLogbin_Precip_total')
da_histNLogbin_Precip_total = xr.DataArray(histNLogbin_Precip_total, dims=('lat_coarse','lon_coarse','Xbin'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse,'Xbin':binplot_Xlog1}, name='histNLogbin_Precip_total')

da_fit_PL_total = xr.DataArray(fit_PL_total, dims=('lat_coarse','lon_coarse'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse}, name='fit_PL_total')
da_fit_tau_total = xr.DataArray(fit_tau_total, dims=('lat_coarse','lon_coarse'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse}, name='fit_tau_total')

da_X_fit_total = xr.DataArray(X_fit_total, dims=('lat_coarse','lon_coarse','Xbin'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse,'Xbin':binplot_Xlog1}, name='X_fit_total')
da_Y_fit_total = xr.DataArray(Y_fit_total, dims=('lat_coarse','lon_coarse','Xbin'), coords={'lat_coarse':lat_coarse,'lon_coarse':lon_coarse,'Xbin':binplot_Xlog1}, name='Y_fit_total')

ds = xr.merge([da_histLogbin_Precip_total, da_histNLogbin_Precip_total, da_fit_PL_total, da_fit_tau_total, da_X_fit_total, da_Y_fit_total])

out_dir = '/neelin2020/sqduan/RGMA_Precip_PDFs/'

ds.to_netcdf(path=out_dir+'PrecipPDFsLogbinFinal_Coarsen10x10_2001-2019_mon'+mon_list[0]+mon_list[1]+mon_list[2]+'.nc', mode='w')

Took 4.78 minutes


In [18]:
Precip_total = None
del Precip_total
gc.collect()

15